In [10]:
import pandas as pd

import osmosys.osmosys
import activityinfo
from activityinfo import Client
from activityinfo import FormProcessing
import importlib
import model.modelAI
from activityinfo.id import generate_id
import json
from requests.exceptions import HTTPError
import os.path

importlib.reload(osmosys.osmosys)
importlib.reload(activityinfo.FormProcessing)
importlib.reload(model.modelAI)

<module 'model.modelAI' from 'D:\\Proyectos\\OSMOSYS_2022\\IntegraciónAI\\AI_OsmosysIntegrationScripts\\model\\modelAI.py'>

In [6]:
## parameters
indicatorCodeAI = 'PRVBG2_01'
print(
    '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')
month = 'JUNIO'
month_number = '2022-06'
year = 2022
test = True

--------------------------------------PRVBG2_01------------------------------------------------------


In [7]:
# busco matchs
subformDf = osmosys.osmosys.getMatchSubforms(indicatorCodeAI)
## print(subformDf)
indicatorIdsOsmosys = subformDf.osmosys_indicador_id.unique()
subformSeries = subformDf.iloc[0]
formIdAI = subformSeries.form_id
parentFormIdAI = subformSeries.parent_form_id
indicatorIdAI = subformSeries.indicator_id
print("formIdAI: " + formIdAI)
print("parentFormIdAI: " + parentFormIdAI)
print("indicatorIdAI: " + indicatorIdAI)
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))

formIdAI: cpahpzql4q3ommxdj
parentFormIdAI: cqc8ug2l4q3ommxdz
indicatorIdAI: c4g1gcukylzqu4n4l
indicatorIdsOsmosys: [130]


In [8]:
## consulto los datos reportados
reportedDf = osmosys.osmosys.getIesPartnerCantonsByIndicatorsIdsOsmosysAndMonth(indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                                month=month, year=year)
print('reported data: ' + str(reportedDf.shape[0]))

reported data: 23


In [9]:
parentFormsIds = FormProcessing.getParentsIds(parentFormIdAI=parentFormIdAI, reportedDf=reportedDf)

In [12]:
## contruyo las estructuras de datos
changesList = []
newIds = []
for index, row in parentFormsIds.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code

    poblacion_meta = ['Refugiados/as y migrantes', 'Comunidad de acogida']
    modalidad_impl = 'cmo12q1kykwwosjka'  # servicios

    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)
    dfDiversidad = osmosys.osmosys.getRefLgbtiDiscapacitadosValues(year=year, month=month, orgOsmosys=orgAcron,
                                                                   indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                   cantonCode=cantonCode)

    dfCa = osmosys.osmosys.getCAValues(year=year, month=month, orgOsmosys=orgAcron,
                                       indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)

    PRVBG2_01_RM_NA = int(df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    PRVBG2_01_RM_NN = int(df.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    PRVBG2_01_RM_MM = int(df.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    PRVBG2_01_RM_HH = int(df.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)
    PRVBG2_01_RM_OTR = int(df.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)
    PRVBG2_01_RM_LGBT = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)
    PRVBG2_01_RM_DS = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'DISCAPACITADOS') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)
    PRVBG2_01_CA_NA = int(dfCa.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    PRVBG2_01_CA_NN = int(dfCa.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    PRVBG2_01_CA_MM = int(dfCa.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    PRVBG2_01_CA_HH = int(dfCa.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)
    PRVBG2_01_CA_OTR = int(dfCa.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)
    PRVBG2_01_CA_LGBT = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'ECUADOR')].iloc[0].value_a)
    PRVBG2_01_CA_DS = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'DISCAPACITADOS') & (
            dfDiversidad['country_of_origin'] == 'ECUADOR')].iloc[0].value_a)
    PRVBG2_01_CUAL = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                                   indicatorsIdsOmosys=indicatorIdsOsmosys).iloc[
        0].value_a

    subform = model.modelAI.SubFormPRVBG2_01(
        mes=month_number,
        colltmgkykvhxgij6=indicatorIdAI,
        rmrp='Si',
        covid='No',
        poblacion_meta=poblacion_meta,
        modalidad_impl=modalidad_impl,
        PRVBG2_01_RM_NA=PRVBG2_01_RM_NA,
        PRVBG2_01_RM_NN=PRVBG2_01_RM_NN,
        PRVBG2_01_RM_MM=PRVBG2_01_RM_MM,
        PRVBG2_01_RM_HH=PRVBG2_01_RM_HH,
        PRVBG2_01_RM_OTR=PRVBG2_01_RM_OTR,
        PRVBG2_01_RM_LGBT=PRVBG2_01_RM_LGBT,
        PRVBG2_01_RM_DS=PRVBG2_01_RM_DS,
        PRVBG2_01_CA_NA=PRVBG2_01_CA_NA,
        PRVBG2_01_CA_NN=PRVBG2_01_CA_NN,
        PRVBG2_01_CA_MM=PRVBG2_01_CA_MM,
        PRVBG2_01_CA_HH=PRVBG2_01_CA_HH,
        PRVBG2_01_CA_OTR=PRVBG2_01_CA_OTR,
        PRVBG2_01_CA_LGBT=PRVBG2_01_CA_LGBT,
        PRVBG2_01_CA_DS=PRVBG2_01_CA_DS,
        PRVBG2_01_CUAL=PRVBG2_01_CUAL
    )

    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)

print('changes to charge: ' + str(len(changesList)))

changes to charge: 23


In [13]:
## creo respaldo
changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)
#open text file
try:
    os.mkdir(month)
except Exception:
    pass
text_file = open(os.path.join(month,"data_" + indicatorCodeAI + ".json"), "w")
#write string to file
n = text_file.write(finalJson)

#close file
text_file.close()
print(" creado respaldo: "+text_file.name)
newIdsDict= { "newIds": newIds}
newIdsDf=pd.DataFrame(newIdsDict)
newIdsDf.to_csv(os.path.join(month,"new_ids_" + indicatorCodeAI + ".csv"))

 creado respaldo: JUNIO\data_PRVBG2_01.json


In [14]:
## envio a AI
if (test):
    print('--------------------------------------'+indicatorCodeAI+'------------------------------------------------------')
else:
    try:
        if len(changesList) > 0:
            print('se envia a AI')
            client = Client(token=osmosys.osmosys.getToken(), base_url='https://www.activityinfo.org/resources')
            client.post_resource(path='update', body=finalJson)
            print(
                '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')
        else:
            print('nada que enviar a AI')
    except HTTPError as e:
        code = e.response.status_code
        print('error:')
        print(code)

        print(e)

--------------------------------------PRVBG2_01------------------------------------------------------
